# **Dataset Preparation**

In [2]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 6.8 MB/s 
     |████████████████████████████████| 132 kB 65.4 MB/s 
     |████████████████████████████████| 182 kB 52.0 MB/s 
     |████████████████████████████████| 213 kB 70.4 MB/s 
     |████████████████████████████████| 127 kB 68.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 8.1 MB/s 
     |████████████████████████████████| 7.6 MB 56.4 MB/s 


In [3]:
from datasets import load_dataset,Dataset

dataset = load_dataset("csebuetnlp/xlsum","english")
dataset_test = dataset.get("test")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset xlsum downloaded and prepared to /root/.cache/huggingface/datasets/csebuetnlp___xlsum/english/2.0.0/518ab0af76048660bcc2240ca6e8692a977c80e384ffb18fdddebaca6daebdce. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
goverment_politic_key = ["goverment", "politic", "election","political","parties", "political party"]
health_key = ["covid-19", "corona", "health", "covid", "vaccine", "medicine", "aids", "hiv", "virus"]
economic_key = ["economy", "economic", "recession", "inflation", "economical"]
infrastucture_development_key = ["infrastructure", "facilities", "train", "development","construction"]
crime_key = ["crime", "robbery", "theft", "sexual harassement","illegal", "drug","criminal" "law","criminal"]
tech_industry_key = ["tech", "technology", "industry", "startup", "start-up", "business"]

In [5]:
import pandas as pd
import re

def insert_new_data(key,df_source,df_new,col):
  df_pd_new = df_new
  idx=1
  for value in df_source:
    key_found = False
    for k in key:
      if re.findall(k, value[col]):
        key_found = True
        break 
    if key_found:
      new_value = pd.DataFrame({"id":value.get("id") ,"url":value.get("url"), "title":value.get("title"), "text": value.get("text"), "summary": value.get("summary")},index=[idx])
      idx = idx +1
      df_pd_new = df_pd_new.append(new_value)
    if (idx == 6):
      break

  return df_pd_new

In [6]:
import copy
from datasets import Dataset
df_train_xlsum = copy.deepcopy(dataset["test"])

In [7]:
# Crime
df = pd.DataFrame(columns=['id', 'url', 'title', 'summary', 'text']) 
df_test_crime = insert_new_data(crime_key, df_train_xlsum,df,"text")
print(f'lenght dataset crime                : {len(df_test_crime)}')

# Economic
df = pd.DataFrame(columns=['id', 'url', 'title', 'summary', 'text']) 
df_test_economic = insert_new_data(economic_key, df_train_xlsum,df,"text")
print(f'lenght dataset economic             : {len(df_test_economic)}')

lenght dataset crime                : 5
lenght dataset economic             : 5


In [8]:
df_test_economic

,id,url,title,summary,text
1,science-environment-44141840,https://www.bbc.com/news/science-environment-4...,How physics gender gap starts in the classroom,Some progress has been made in encouraging gir...,"By Helen BriggsBBC News In 2016, 1.9% of girls..."
2,business-47891982,https://www.bbc.com/news/business-47891982,Brexit: CBI boss says agree deal or hold secon...,"The president of business organisation, the CB...",John Allan stressed this was his personal view...
3,entertainment-arts-26706229,https://www.bbc.com/news/entertainment-arts-26...,BBC aiming to put arts in mainstream,The BBC is to feature more arts stories in pro...,"On Tuesday, the corporation's director general..."
4,business-38842721,https://www.bbc.com/news/business-38842721,Has the 'Brexit effect' disappeared?,"Hang out the bunting - well, at least buy a co...",Kamal AhmedEconomics editor@bbckamalon Twitter...
5,uk-politics-12657524,https://www.bbc.com/news/uk-politics-12657524,David Cameron says enterprise is only hope for...,"With no money left in the government coffers, ...","By Brian WheelerPolitical reporter, BBC News I..."


# **Model Download**

In [9]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM, EvalPrediction

## **T5 Small**

In [30]:
model_t5small_without_augmentation = AutoModelForSeq2SeqLM.from_pretrained("fanzru/t5-small-finetuned-xlsum-10-epoch")
tokenizer_t5small_without_augmentation = AutoTokenizer.from_pretrained("fanzru/t5-small-finetuned-xlsum-10-epoch")

In [31]:
model_t5small_with_augmentation = AutoModelForSeq2SeqLM.from_pretrained("fanzru/t5-small-finetuned-xlsum-with-multi-news-10-epoch")
tokenizer_t5small_with_augmentation = AutoTokenizer.from_pretrained("fanzru/t5-small-finetuned-xlsum-with-multi-news-10-epoch")

## **BART BASE**

In [32]:
model_bart_with_augmentation = AutoModelForSeq2SeqLM.from_pretrained("fanzru/bart-base-finetuned-xlsum-with-multi-news-10-epoch")
tokenizer_bart_with_augmentation = AutoTokenizer.from_pretrained("fanzru/bart-base-finetuned-xlsum-with-multi-news-10-epoch")

In [33]:
model_bart_without_augmentation = AutoModelForSeq2SeqLM.from_pretrained("fanzru/bart-base-finetuned-xlsum-10-epoch")
tokenizer_bart_without_augmentation = AutoTokenizer.from_pretrained("fanzru/bart-base-finetuned-xlsum-10-epoch")

# **Generate Data Human Evaluation**

In [62]:
import nltk
nltk.download('punkt')

t5_prefix = "summarize: "
bart_prefix = ""

def summarize(model, tokenizer,prefix, text):
  inputs = ["summarize: " + text]
  inputs = tokenizer(inputs, max_length=1024, truncation=True, return_tensors="pt")
  output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=40, max_length=128)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]
  return predicted_title

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [63]:
def dataframe_summarize(df):
  for idx in range(len(df)):

    # summarize t5 small without augmentation
    summary_t5small_without_augmentation = summarize(model_t5small_without_augmentation,tokenizer_t5small_without_augmentation,t5_prefix, df.loc[idx]["text"])
    # summarize t5 small with augmentation
    summary_t5small_with_augmentation = summarize(model_t5small_with_augmentation,tokenizer_t5small_with_augmentation,t5_prefix,df.loc[idx]["text"])
    # summarize bart base without augmentation
    summary_bartbase_without_augmentation = summarize(model_bart_with_augmentation,tokenizer_bart_with_augmentation,bart_prefix,df.loc[idx]["text"])
    # summarize bart base with augmentation'
    summary_bartbase_with_augmentation = summarize(model_bart_without_augmentation,tokenizer_bart_without_augmentation,bart_prefix, df.loc[idx]["text"])

    df.loc[idx,"summary_t5small_without_augmentation"] = summary_t5small_without_augmentation
    df.loc[idx,"summary_t5small_with_augmentation"] = summary_t5small_with_augmentation
    df.loc[idx,"summary_bartbase_without_augmentation"] = summary_bartbase_without_augmentation
    df.loc[idx,"summary_bartbase_with_augmentation"] = summary_bartbase_with_augmentation

  return df

In [39]:
df_eval = pd.concat([df_test_economic, df_test_crime], ignore_index=True)

In [65]:
df_with_summary = dataframe_summarize(df_eval)

In [64]:
df_with_summary.to_excel("summary_with_earlystoping.xlsx")

In [61]:
df_with_summary.loc[3]["summary_bartbase_with_augmentation"]

"It is one of the most important events in the history of the Bank of England's Monetary Policy Committee."